## Optimization exercise

**Goal: Train the 2nd order polynomial predictor using gradient descent. Optimize the stepsizes and compare against scikit-learn's implementation.**

In [1]:
# IMPORT PACKAGES
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [2]:
data_path = '../data/'
nb = pd.read_csv(data_path+'nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']]
y = nb['PTS']

#### Task I
Download the data from [here](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing&resourcekey=0-16O9Fc5eaJH99-M7AHqHOg).

In [3]:
nb.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22015,1610612750,MIN,Minnesota Timberwolves,21501226,2016-04-13,MIN vs. NOP,W,240,144,...,0.826,5,38,43,41,14,8,13,20,35.0
1,22015,1610612749,MIL,Milwaukee Bucks,21501225,2016-04-13,MIL vs. IND,L,240,92,...,0.846,7,36,43,23,8,3,15,15,-5.0
2,22015,1610612738,BOS,Boston Celtics,21501217,2016-04-13,BOS vs. MIA,W,240,98,...,0.864,10,29,39,20,7,3,7,20,10.0
3,22015,1610612747,LAL,Los Angeles Lakers,21501228,2016-04-13,LAL vs. UTA,W,239,101,...,0.867,8,39,47,19,6,3,13,17,5.0
4,22015,1610612739,CLE,Cleveland Cavaliers,21501220,2016-04-13,CLE vs. DET,L,265,110,...,0.733,8,35,43,21,4,7,10,23,-2.0



#### TASK II
Create a function `psi(x)`, which transforms features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features (add each feature squared and each pair of features multiplied with every other)

**Input:** DataFrame x from above. It contains columns AST, REB, STL

**Output:** DataFrame with columns: AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB. The number of rows should be the same as Input.

In [8]:
def psi(x):
    # Select the AST, REB, and STL columns from the input DataFrame
    selected_columns = ['AST', 'REB', 'STL']
    x = x[selected_columns]

    # Create a PolynomialFeatures object for 2nd order (degree=2) polynomials
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)

    # Fit and transform the selected columns to generate the polynomial features
    x_poly = poly.fit_transform(x)

    # Create a DataFrame with the desired column names
    column_names = selected_columns + ['1'] + [f"{col}^2" for col in selected_columns] + [f"{col1}{col2}" for col1 in selected_columns for col2 in selected_columns if col1 <= col2]
    x_poly_df = pd.DataFrame(x_poly, columns=column_names)

    return x_poly_df

# Example usage:
# Assuming 'data' is your DataFrame with AST, REB, and STL columns
transformed_data = psi(nb[['AST', 'REB', 'STL']])


ValueError: Shape of passed values is (7380, 9), indices imply (7380, 13)

#### Task III
Create a transformed data matrix **X**, where each **x** is mapped to psi(x).

HINT: We need to apply our function from Task II to matrix (DataFrame) x

In [ ]:
def psi(x):
    # Select the AST, REB, and STL columns from the input DataFrame
    selected_columns = ['AST', 'REB', 'STL']
    x = x[selected_columns]

    # Create a PolynomialFeatures object for 2nd order (degree=2) polynomials
    poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)

    # Fit and transform the selected columns to generate the polynomial features
    x_poly = poly.fit_transform(x)

    # Create a DataFrame with the desired column names
    columns = selected_columns + [f"{col}^2" for col in selected_columns] + [f"{col1}{col2}" for col1 in selected_columns for col2 in selected_columns if col1 <= col2]
    x_poly_df = pd.DataFrame(x_poly, columns=columns)

    # Add a column of ones (for the intercept term)
    x_poly_df['1'] = 1

    return x_poly_df

# Example usage:
# Assuming 'data' is your DataFrame with AST, REB, and STL columns
transformed_data = psi(data)


#### Task IV
Create a function `p2(x,w)`, which outputs the value of the polynomial ata given row of `x` for given parameters `w`.

Inputs:
- x: DataFrame from above
- w: vector which represents beta coeficients for each column of X from the Task III.

Ouputs:
- Series of the same length as DataFrame x. Each value is a dot product of particular row in DataFrame and coeficients `w`.


HINT:
- length of w needs to be the same as number of columns of the dataframe that is returned from function `psi(x)`

Example Input:

`p2(x, [0.06, 0.05,0.03,0.01,0.02,0.02,0.04, 0.03,0.02,0.01])`

Example Output:

```
0       130.37
1        76.19
2        61.21
3        74.51
4        64.97
         ...  
7375     63.01
7376     79.59
7377     97.25
7378     78.85
7379     61.53
Length: 7380, dtype: float64

```

Our columns in the DataFrame **X** that is the output of `psi(x)` were in this order: `AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB`. If your columns are in different order the result can be different even for the **w**.

#### Task V
Create a function `Loss(x,y,w)`, which computes the squared loss of predicting **y** from **x** by `p2(x,w)` using parameters **w**. We have specified **y** as the variable PTS above. We will predict scored points (PTS) based on assists, rebounds and steals.


HINTS: 
- Output of `p2(x,w)` represents our predictions. `y_pred = p2(x,w)`
- Loss can be computed as:

```
np.mean((y_pred - y)**2)
```

#### Task VI
Code up the gradient descent. It should input **x**, target variable **y**, stepsize **alpha**, coeficients **w** and **maximum number of iterations**.

Steps:
1. transform input `x`
2. compute initial loss for given, x,y and w and append it to the empty list.
3. Inside the for loop, update each element of **w**, **w[i]**, using gradient descent.

HINT: `w[i] = w[i] - alpha * (1.0 / m) * errors_i.sum()` where `errors_i = (y_pred - y) * X.iloc[:, i]`. We are scaling the errors by multiplicating with values that are relevant for coeficients `w[i]` (column `i` of DataFrame `X`, output of `psi(x)`).

4. compute new loss using updated `w` and append to the list that we created in the step 2.
5. repeat steps 3 and 4 for max number of iterations times.

#### Task VII
Choose an arbitrary **w** and **alpha** and run gradient descent for 100 iterations. How does the loss behave? Does it converge to something?


#### Task VIII
Can you find which **alpha** from `[1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]` has the smallest loss after 100 iterations for a given **w**?